<h1><center> Data Cleaning File  <br><br> 
<font color='grey'> Cleaning Emissions and Traffic Data <br><br>


In [1]:
#setting libraries
import pandas as pd

In [26]:
#reading in emissions data files
emissions_16 = pd.read_excel("2016_emissions.xlsx",
                            sheet_name = "Direct Emitters", #only picks first sheet
                            header = 3,
                            usecols = ["Facility Id", "City", "State", "Zip Code", "County", "Latitude", "Longitude", "Industry Type (subparts)", "Industry Type (sectors)", "Total reported direct emissions"])
emissions_13 = pd.read_excel("2013_emissions.xlsx",
                            sheet_name = "Direct Emitters", #only picks first sheet
                            header = 3,
                            usecols = ["Facility Id", "City", "State", "Zip Code", "County", "Latitude", "Longitude", "Industry Type (subparts)", "Industry Type (sectors)", "Total reported direct emissions"])
emissions_19 = pd.read_excel("2019_emissions.xlsx",
                            sheet_name = "Direct Emitters", #only picks first sheet
                            header = 3,
                            usecols = ["Facility Id", "City", "State", "Zip Code", "County", "Latitude", "Longitude", "Industry Type (subparts)", "Industry Type (sectors)", "Total reported direct emissions"])

In [30]:
#create column names
new_col_names = ["facility_id", "city", "state", "zipcode", "county", "latitude", "longitude", "industry_sub", "industry_main", "total_emissions"]

In [32]:
#rename columns
for df in [emissions_16, emissions_13, emissions_19]:
    df.columns = new_col_names

In [55]:
#save list of relevant states and industry
rel_states = ["DC", "MD", "VA"]
rel_indust = "MN|NN|C|W"

In [64]:
#filter for dc, md, and va + relevant industries
emissions_16 = emissions_16.query('state in @rel_states & industry_sub.str.contains(@rel_indust, na = False)')
emissions_13 = emissions_13.query('state in @rel_states & industry_sub.str.contains(@rel_indust, na = False)')
emissions_19 = emissions_19.query('state in @rel_states & industry_sub.str.contains(@rel_indust, na = False)')

In [67]:
#checking number of rows
[df.shape for df in [emissions_16, emissions_13, emissions_19]]

[(159, 10), (157, 10), (156, 10)]

In [73]:
#adding identifers before merging
emissions_13.loc[:, "time"] = 2013
emissions_16.loc[:, "time"] = 2016
emissions_19.loc[:, "time"] = 2019

In [74]:
emissions_13

,facility_id,city,state,zipcode,county,latitude,longitude,industry_sub,industry_main,total_emissions,time
34,1006971,APG,MD,21005,HARFORD COUNTY,39.395404,-76.307730,C,Other,33282.086,2013
70,1000627,CHESTER,VA,23836,CHESTERFIELD COUNTY,37.339630,-77.288400,C,Other,45318.778,2013
71,1000630,HOPEWELL,VA,23860,HOPEWELL CITY,37.181500,-77.162100,"C,G,PP","Chemicals,Suppliers of CO2",1115670.506,2013
79,1001223,CUMBERLAND,MD,21502,Allegany,39.594400,-78.745600,"C,D,PP","Power Plants,Suppliers of CO2",1482156.434,2013
152,1007291,MARRIOTTSVILLE,MD,21104,HOWARD COUNTY,39.307180,-76.898620,"C,HH",Waste,32301.468,2013
...,...,...,...,...,...,...,...,...,...,...,...
7029,1008708,HOPEWELL,VA,23860,HOPEWELL CITY,37.302012,-77.264354,"AA,C",Pulp and Paper,167627.162,2013
7039,1000256,COVINGTON,VA,24426,Alleghany,37.799700,-79.994600,"AA,C,TT","Pulp and Paper,Waste",1597893.400,2013
7057,1004969,PORTSMOUTH,VA,23704,PORTSMOUTH CITY,36.808889,-76.304251,C,Waste,226965.774,2013
7178,1001205,YORKTOWN,VA,23692,York,37.214400,-76.461100,"C,D",Power Plants,845112.354,2013
